# Anwendung des Classifers

## Laden von Paketen

 Die Anwendung des Classifiers ist so einfach wie möglich gestaltet. In einem ersten Schritt wählen wir unser Enviorment `future-skill-classification` und laden die für die Vorhersage der Labels nötigen Pakete:

In [ ]:
import pyreadr  # Zum Lesen von R-Daten in Python
import os       # Funktionen zum Interagieren mit dem Betriebssystem
import pandas as pd  # Datenanalyse und -manipulation mit DataFrames
import re # Regrex-Paket für die Bereinigung von Strings
from tqdm import tqdm # Paket fürs Anzeigen von Fortschritt beim Ausführen von Funktionen
from setfit import SetFitModel  # Zum Laden des SetFit-Modells von Hugging Face bzw. aus dem lokalen Ordner heraus


## Laden der HEX-Daten

Wir checken das Root-Verzeichnis. Stimmt dieses nicht, bestimmen wir das Korrekte und lesen `db_hex.rds` ein. 

In [ ]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')
# lade HEX-Daten. Hinweise: Zurückgegben eine Liste mit einem Element (dem Data Frame).
# Dieses lassen wir uns mit `.values())[0]` zurückgeben.
df_hex = list(pyreadr.read_r('data/db_hex.rds').values())[0]

Um die Classifier zu testen, wenden wir ihn vorerst nur auf 10.000 zufällig gezogene Zeilen an. Dafür *ziehen* wir die entsprechende Stichprobe und speichern sie in dem Objekt `df_hex_sample`.

In [ ]:
df_hex_sample = df_hex.sample(n=10000)

In einem weiteren Schritt werden die Daten für die Klassifikation vorbereitet: 

- `NAs` werden durch leere Strings ersetzt
- Die Variablen `titel` und `kursbeschreibung` zusammengefügt und – falls eine Kursbeschreibung vorhanden ist – ein Doppelpunkt zwischen `titel` und `kursbeschreibung` gesetzt.

In [ ]:
# Ersetze NaN-Werte in den Spalten 'titel' und 'kursbeschreibung' durch leere Strings
df_hex_sample['titel'] = df_hex_sample['titel'].fillna('')
df_hex_sample['kursbeschreibung'] = df_hex_sample['kursbeschreibung'].fillna('')

# Kombiniere die Spalten und füge den Doppelpunkt nur hinzu, wenn eine Kursbeschreibung vorhanden ist
df_hex_sample['sentence'] = df_hex_sample.apply(lambda row: row['titel'] + (": " + row['kursbeschreibung'] if row['kursbeschreibung'] else ""), axis=1)

## Definition von Funktionen für die Vorhersage von Future Skills

Um die Future Skills der Kurse der Datenbank mit unserem Modell vorhersagen zu können, erstellen wir eine Funktion, die durch die Zeilen der `db_hex.rds` iteriert und die entsprechenden Schätzungen anhand der oben präparierten `sentence`-Variable vornimmt.

In [ ]:
# Fortschrittsbalken für Pandas aktivieren, um die Fortschritt der Prädiktion besser nachvollziehen zu können.
tqdm.pandas()

# Definiere eine Funktion, die das Modell auf eine Kursbeschreibung anwendet
def predict_course_description(description):
    # Überprüfen, ob die Beschreibung ein String ist
    if isinstance(description, str):
        preds = model(description)
        return preds
    # Falls `sentence` ein nicht-String-Objekt, gib eine leere Liste
    return []

Die Funktion `predict_course_description` gibt uns einen sechsdimensionalen Tensoren zurück, der die Existenz der Future Skills pro Kurs anzeigt:

```
[1, 0, 0, 1, 0, 0]
```
Für die Interpretation des Tensors berücksichtigen wir die Reihenfolge der Variablen unserer Eingangsdaten. Siehe dazu in  `notebooks/Tiny_Few_Shot_Multi_Label_Classifier.ipynb` Sektion `Daten laden`. Im derzeitigen Fall wäre dies die folgende:

```
['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing']
```

Dementsprechend würden in dem obigen Beispiel die Skills 'Data Analytics & KI' sowie 'IT-Architektur' klassifiziert.

Um die Tensoren zeilenweise wieder in gültige Labels zu überführen spezifizieren wir eine weitere Funktion: 

In [ ]:
fs_labels = ['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing']

import pandas as pd
import numpy as np

def convert_tensor_to_labels(tensor):
    # Falls der Tensor NaN ist, gib None zurück
    if pd.isna(tensor).all():  # Überprüft, ob alle Werte im Tensor NaN sind
        return None  # Oder verwende np.nan
    
    # Sicherstellen, dass die Länge des Tensors der Länge der fs_labels-Liste entspricht
    if len(tensor) != len(fs_labels):
        print(f"Warnung: Unerwartete Tensorgröße {len(tensor)}, erwartet: {len(fs_labels)}")
        return 'Fehlerhafte Vorhersage'
    
    # Identifiziere die Positionen, wo der Wert 1 ist
    selected_labels = [fs_labels[i] for i, val in enumerate(tensor) if val == 1]
    
    # Falls keine Labels ausgewählt wurden, gib None zurück
    return ', '.join(selected_labels) if selected_labels else None  # Oder np.nan

## Modell laden, Prädiktion durchführen

Das trainierte Modell kann nun entweder lokal oder aus dem Hugging Face-Hub geladen werden. Anschließend wird mit `predict_course_description` die Prädiktion pro Kurs durchgeführt und die resultierenden Tensoren in eine leicht interpretierbare String-Variable überführt.

In [ ]:
# Laden des vortrainierten Modells

# Aus dem HF-HUB:
#model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")

# Lokal:
model = SetFitModel.from_pretrained("models")  # Ändere den Pfad zu deinem Modell

# Prediction anhand von predict_course_description
df["Pred_Tensor"] = df["sentence"].progress_apply(predict_course_description)

# Umwandlung des Tensors in String-Variable
df["FS_Skill"] = df["Pred_Tensor"].progress_apply(lambda x: convert_tensor_to_labels(x))

# Tabelle der Resultate:
label_counts = df['FS_Skill'].value_counts()

## Datenexport

Done🤗. Insbesondere bei der Klassifizierung der ganzen HEX Datenbank exportieren wir aufgrund der überaus hohen Menge an Zeilen nach `.csv`.

In [ ]:
# Exportiere den DataFrame in eine Excel-Datei
df.to_csv('daten/hex_classified_fs.csv', index=False)